# GOLD LAYER SCRIPT

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS silver;

CREATE OR REPLACE TABLE silver.healthcare
USING DELTA
AS
SELECT *
FROM delta.`abfss://silver@sthealthcarede.dfs.core.windows.net/Healthcare_Dataset/`;

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS gold;

CREATE OR REPLACE TABLE gold.dim_patient
USING DELTA
AS
SELECT DISTINCT
    sha2(concat_ws('||', name, gender), 256) AS patient_key,
    name,
    gender,
    blood_type
FROM silver.healthcare;

CREATE OR REPLACE TABLE gold.dim_doctor
USING DELTA
AS
SELECT DISTINCT
    sha2(concat_ws('||', doctor), 256) AS doctor_key,
    doctor,
    hospital
FROM silver.healthcare;

CREATE OR REPLACE TABLE gold.dim_hospital
USING DELTA
AS
SELECT DISTINCT
    sha2(hospital_clean, 256) AS hospital_key,
    hospital_clean AS hospital_name
FROM silver.healthcare;

CREATE OR REPLACE TABLE gold.dim_insurance
USING DELTA
AS
SELECT DISTINCT
    sha2(insurance_provider, 256) AS insurance_key,
    insurance_provider
FROM silver.healthcare;

CREATE OR REPLACE TABLE gold.dim_date
USING DELTA
AS
SELECT
    CAST(date_format(d, 'yyyyMMdd') AS INT) AS date_key,
    d AS full_date,
    YEAR(d) AS year,
    MONTH(d) AS month_num,
    DAY(d) AS day,
    QUARTER(d) AS quarter
FROM (
    SELECT explode(
        sequence(
            DATE('2019-05-08'),
            DATE('2024-05-07'),
            INTERVAL 1 DAY
        )
    ) AS d
);

CREATE OR REPLACE TABLE gold.fact_admissions
USING DELTA
AS
SELECT
    s.admission_key,
    -- Dimension foreign keys
    p.patient_key,
    d.doctor_key,
    h.hospital_key,
    i.insurance_key,
    -- Date keys
    CAST(date_format(s.date_of_admission, 'yyyyMMdd') AS INT) AS admission_date_key,
    CAST(date_format(s.discharge_date, 'yyyyMMdd') AS INT) AS discharge_date_key,
    -- Measures
    s.billing_amount,
    s.length_of_stay,
    -- Event attributes
    s.admission_type,
    s.room_number,
    s.medication,
    s.test_results,
    -- Metadata
    s.ingestion_date
FROM silver.healthcare s
JOIN gold.dim_patient p ON sha2(concat_ws('||', s.name, s.gender), 256) = p.patient_key
JOIN gold.dim_doctor d ON sha2(s.doctor, 256) = d.doctor_key
JOIN gold.dim_hospital h ON sha2(s.hospital_clean, 256) = h.hospital_key
JOIN gold.dim_insurance i ON sha2(s.insurance_provider, 256) = i.insurance_key;
